In [39]:
import cv2
import numpy as np


In [40]:
from keras.models import load_model

model = load_model('fer_model.h5')
model.load_weights('./face_emotion_model.h5', by_name=True)
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def predict_emotion(image):
	return emotions[np.argmax(model.predict(np.asarray(image)))]

In [41]:
from keras.utils import img_to_array, load_img

model.predict(np.asarray(np.array([img_to_array(load_img('images/train/angry/10.jpg', color_mode="grayscale"))])))

1/1 [==============================] - 1s 1s/step


array([[0.6759712 , 0.01791541, 0.18767276, 0.0023076 , 0.05972439,
        0.04854424, 0.00786433]], dtype=float32)

In [42]:
from keras.utils import img_to_array, load_img

print(predict_emotion(np.array([img_to_array(load_img('images/train/angry/10.jpg', color_mode="grayscale"))])))

1/1 [==============================] - 0s 145ms/step
angry


In [43]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
scale = 1.5
def process_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        x_new = int(x - (scale - 1) / 2 * w)
        y_new = int(y - (scale - 1) / 2 * h)
        w_new = int(scale * w)
        h_new = int(scale * h)

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x_new, y_new), (x_new+w_new, y_new+h_new), (255, 0, 0), 2)

        # Crop the face from the frame
        face_image = gray[y_new:y_new+h_new, x_new:x_new+w_new] 

        try:
            face_image = cv2.resize(face_image, (48, 48))
            emo = predict_emotion(face_image.reshape(1, 48, 48, 1))
            face_image = cv2.cvtColor(face_image, cv2.COLOR_GRAY2BGR)
            cv2.putText(frame, emo, (int(x_new+w_new/3), y_new+h_new+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        except Exception as e:
            # print(e)
            continue
        cv2.imshow('Cropped Face', face_image)
    return frame

In [44]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        frame = process_faces(frame)
        cv2.imshow('Face Detection Demo', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 53ms/step
